In [17]:
import pandas as pd 
import requests 
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qsl, urlunparse, urlencode
import os 


In [4]:
url = "https://trafficdata.tii.ie/tfmonthreport.asp?sgid=XzOA8m4lr27P0HaO3_srSB&spid=256365229484&reportdate=2021-10-27&enddate=2021-10-27"
document = requests.get(url, timeout=30)
html_doc = document.content
soup = BeautifulSoup(html_doc, 'html.parser')

In [26]:
from requests.models import PreparedRequest
url = "https://trafficdata.tii.ie/tfmonthreport.asp?sgid=XzOA8m4lr27P0HaO3_srSB&spid=256365229484"
params = {'reportdate': '2020-10-20'}
req = PreparedRequest()
req.prepare_url(url, params)
print(req.url)




https://trafficdata.tii.ie/tfmonthreport.asp?sgid=XzOA8m4lr27P0HaO3_srSB&spid=256365229484&reportdate=2020-10-20


In [16]:
def _get_siteId(soup):
	siteId_ele = soup.select('div#content #subtitlebar span')[1]
	siteId_text = siteId_ele.text.split(":")[1].strip()
	siteId_text = str(int(siteId_text))
	return siteId_text


In [18]:
def _get_monthly_data(soup):
	list_tr_element = soup.select("table#gridTable tr")
	content = []
	for i in range(1, len(list_tr_element)):
		tr_element = list_tr_element[i]
		if i == 1:
			# header 
			ths = tr_element.select('th')
			for j in range(len(ths)):
				content.append([ths[j].text])
		else:
			tds = tr_element.select('td')
			for j in range(len(tds)):
				content[j].append(tds[j].text)

	# convert content to pandas dataframe
	df = pd.DataFrame(content)
	df = df.transpose()
	new_header = df.iloc[0]
	df = df[1:]
	df.columns = new_header

	df.index = df['Time']
	df = df.drop(columns=['Time'], axis=1)
	return df

In [20]:

SAVE_FOLDER = os.path.join(os.getcwd(), "craw_data")
def get_montly_report(url):
	parsed_url = urlparse(url)
	query = parse_qs(parsed_url.query)
	report_date = query['reportdate'][0]
	document = requests.get(url, timeout=30)
	html_doc = document.content
	soup = BeautifulSoup(html_doc, 'html.parser')
	siteId_text = _get_siteId(soup)
	df = _get_monthly_data(soup)
	# save it to csv file 
	outfile_folfer = os.path.join(SAVE_FOLDER, report_date)
	if not os.path.exists(outfile_folfer):
		os.makedirs(outfile_folfer)
	df.to_csv(os.path.join(outfile_folfer, siteId_text + ".csv"))

url = "https://trafficdata.tii.ie/tfmonthreport.asp?sgid=XzOA8m4lr27P0HaO3_srSB&spid=256365229484&reportdate=2021-10-27&enddate=2021-10-27"
get_montly_report(url)
